# Create Network Dataset and Estimate the Optimal Route

### Connect your GIS (optional)

In [1]:
from arcgis.gis import GIS
gis = GIS('home')

The road data used to create the network dataset was sourced from OpenStreetMap, but it can also come from any other open-source road data.  

### Create road network dataset

Before building the network, ensure that the road attributes contain necessary information; otherwise, you may need to create a new column. In this project, I will generate modes that include Distance, Time, and Risk. I will add columns for length (km), time (min), and high-risk values.

In [ ]:
# Calculate road length (km)
arcpy.management.CalculateGeometryAttributes(
    in_features="osm_road",
    geometry_property="Length_km LENGTH_GEODESIC",
    length_unit="KILOMETERS",
    area_unit="",
    coordinate_system='GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]',
    coordinate_format="SAME_AS_INPUT"
)

In [ ]:
# Calculate time (min)
arcpy.management.CalculateField(
    in_table="osm_road",
    field="MINUTES",
    expression="!Length! / !maxspeed! * 60",
    expression_type="PYTHON3",
    code_block="",
    field_type="TEXT",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

In [ ]:
# Incorporate the results from the high-risk area analysis (obtained from the "Analyzing the High-Risk Area" notebooks) into the road dataset.
arcpy.sa.AddSurfaceInformation(
    in_feature_class="osm_road",
    in_surface="Reclass_risk2",
    out_property="Z_MEAN",
    method="BILINEAR",
    sample_distance=None,
    z_factor=1,
    pyramid_level_resolution=0,
    noise_filtering=""
)

Create the Network Dataset then Build Network

In [ ]:
arcpy.na.CreateNetworkDataset(
    feature_dataset=r"D:\fall2023\arc1\project\Network\Network.gdb\ND",
    out_name="Driving",
    source_feature_class_names="osm_road",
    elevation_model="ELEVATION_FIELDS"
)

In [ ]:
arcpy.na.BuildNetwork(
    in_network_dataset=r"D:\fall2023\arc1\project\Network\Network.gdb\ND\Driving",
    force_full_build="NO_FORCE_FULL_BUILD"
)

### Generate random points to identify incident locations

To identify incident locations, I generate 100 random points, create drive-time buffers for each point, and then utilize the Google API to request information on nearby fire stations.

create 100 random points

In [ ]:
arcpy.management.CreateRandomPoints(
    out_path=r"D:\fall2023\arc1\project\Network\Network.gdb",
    out_name="random_points",
    constraining_feature_class="COUNTY1984",
    constraining_extent="0 0 250 250",
    number_of_points_or_field=100,
    minimum_allowed_distance="0 DecimalDegrees",
    create_multipoint_output="POINT",
    multipoint_size=0
)

Establish drive-time buffers of 5, 10, and 15 minutes for each random point, corresponding to distances of 4.17, 8.33, and 12.5 km, based on a speed of 50 km/hr.

In [ ]:
# 5 mins buffer
arcpy.analysis.Buffer(
    in_features=r"Closest Facility\Incidents",
    out_feature_class=r"D:\fall2023\arc1\project\road_risk\road_risk.gdb\Incidents_Buffer5min",
    buffer_distance_or_field="4.17 Kilometers",
    line_side="FULL",
    line_end_type="ROUND",
    dissolve_option="NONE",
    dissolve_field=None,
    method="GEODESIC"
)

In [ ]:
# 10 mins buffer
arcpy.analysis.Buffer(
    in_features=r"Closest Facility\Incidents",
    out_feature_class=r"D:\fall2023\arc1\project\road_risk\road_risk.gdb\Incidents_Buffer10min",
    buffer_distance_or_field="8.33 Kilometers",
    line_side="FULL",
    line_end_type="ROUND",
    dissolve_option="NONE",
    dissolve_field=None,
    method="GEODESIC"
)

In [ ]:
# 15 mins buffer
arcpy.analysis.Buffer(
    in_features=r"Closest Facility\Incidents",
    out_feature_class=r"D:\fall2023\arc1\project\road_risk\road_risk.gdb\Incidents_Buffer15min",
    buffer_distance_or_field="12.5 Kilometers",
    line_side="FULL",
    line_end_type="ROUND",
    dissolve_option="NONE",
    dissolve_field=None,
    method="GEODESIC"
)

Select a random point and use the Google Places API to request information on nearby fire stations.

In [1]:
import requests
import pandas as pd
import geopandas as gpd # geopandas may can not use in Arcpro notebooks
import json

In [8]:
loc = "22.6236521" + "%2C" + "120.5675280"  # input the random point's coordinate
rad = "10000"  # how far you want to search
keyword = "fire station"  # input any keyword you want to search
api_key = ""  # Your Google Places API key
url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={loc}&radius={rad}&keyword={keyword}&key={api_key}"

response = requests.get(url)
data = response.json()

# Extract fire stations
fire_stations = [result for result in data['results'] if 'fire_station' in result.get('types', [])]

# Create a DataFrame
columns = ['Name', 'Latitude', 'Longitude', 'Address']
data_list = []

for station in fire_stations:
    name = station.get('name', 'Unknown')
    lat = station['geometry']['location']['lat']
    lng = station['geometry']['location']['lng']
    address = station.get('vicinity', 'Unknown')

    data_list.append([name, lat, lng, address])

df = pd.DataFrame(data_list, columns=columns)
df


,Name,Latitude,Longitude,Address
0,Pingtung County Fire Bureau,22.676946,120.498512,"No. 47號, Shengli E Rd"
1,屏東縣政府消防局第一大隊屏東第二分隊,22.638053,120.484698,"No. 52號, Longhua Rd"
2,Pingtung County Fire Bureau,22.693158,120.490289,"No. 226號, Zhongxiao Rd"
3,屏東縣政府消防局 第二大隊泰武分隊,22.592109,120.633653,佳平巷122號
4,Pingtung County Fire Bureau,22.704900,120.644437,風景路1-20號
5,Pingtung County Fire Bureau,22.693113,120.542558,"No. 572號, Zhongxing Rd"
6,屏東縣政府消防局 第二大隊竹田分隊,22.584257,120.545727,"No. 41-20號, Ziqiang Rd"
7,屏東縣政府消防局 第二大隊萬丹分隊,22.588452,120.485291,"913No. 149號, Heping W Rd"
8,屏東縣政府消防局 第二大隊潮州分隊,22.557773,120.543456,"No. 191號, Wenhua Rd"
9,屏東縣政府消防局 第二大隊內埔分隊,22.613773,120.566326,"No. 209號, Guangji Rd"


In [2]:
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Lng, df.Lat), crs="EPSG:4326")
# Convert the GeoDataFrame to GeoJSON
gdf_json = gdf.to_crs("EPSG:4326").to_json()

#save the data
gdf.to_file("fire_station.shp")


### Find the optimal route from fire stations to the incident location.

Utilize the "Risk" mode to avoid high-risk route

In [ ]:
arcpy.na.MakeClosestFacilityAnalysisLayer(
    network_data_source=r"D:\fall2023\arc1\project\Network\Network.gdb\ND\Driving6",
    layer_name="Closest Facility",
    travel_mode="risk", # Depends on which mode you want to use
    travel_direction="TO_FACILITIES",
    cutoff=None,
    number_of_facilities_to_find=1,
    time_of_day=None,
    time_zone="LOCAL_TIME_AT_LOCATIONS",
    time_of_day_usage="START_TIME",
    line_shape="ALONG_NETWORK",
    accumulate_attributes=None,
    generate_directions_on_solve="NO_DIRECTIONS",
    ignore_invalid_locations="SKIP"
)

In [ ]:
# Add fire stations data
arcpy.na.AddLocations(
    in_network_analysis_layer="Closest Facility",
    sub_layer="Facilities",
    in_table="fire_station",
    field_mappings="Name Name #;CurbApproach # 0;Attr_TravelTime # 0;Attr_Length # 0;Attr_risk # 0;Cutoff_TravelTime # #;Cutoff_Length # #;Cutoff_risk # #",
    search_tolerance="20000 Meters",
    sort_field=None,
    search_criteria="osm_road SHAPE;Driving6_Junctions NONE",
    match_type="MATCH_TO_CLOSEST",
    append="APPEND",
    snap_to_position_along_network="NO_SNAP",
    snap_offset="5 Meters",
    exclude_restricted_elements="EXCLUDE",
    search_query=None,
    allow_auto_relocate="ALLOW"
)

In [ ]:
# Add the random point you chose
arcpy.na.AddLocations(
    in_network_analysis_layer="Closest Facility",
    sub_layer="Incidents",
    in_table=r"random_points",
    field_mappings="Name Name #;TargetFacilityCount TargetFacilityCount #;CurbApproach CurbApproach 0;Attr_Minutes # 0;Attr_TravelTime Attr_TravelTime 0;Attr_Miles # 0;Attr_Kilometers # 0;Attr_TimeAt1KPH # 0;Attr_WalkTime # 0;Attr_TruckMinutes # 0;Attr_TruckTravelTime # 0;Cutoff_Minutes # #;Cutoff_TravelTime Cutoff_TravelTime #;Cutoff_Miles # #;Cutoff_Kilometers # #;Cutoff_TimeAt1KPH # #;Cutoff_WalkTime # #;Cutoff_TruckMinutes # #;Cutoff_TruckTravelTime # #",
    search_tolerance="20000 Meters",
    sort_field=None,
    search_criteria="main.Routing_Streets SHAPE",
    match_type="MATCH_TO_CLOSEST",
    append="APPEND",
    snap_to_position_along_network="NO_SNAP",
    snap_offset="5 Meters",
    exclude_restricted_elements="EXCLUDE",
    search_query=None,
    allow_auto_relocate="ALLOW"
)

In [ ]:
# run the network analysis
arcpy.na.Solve(
    in_network_analysis_layer="Closest Facility",
    ignore_invalids="SKIP",
    terminate_on_solve_error="TERMINATE",
    simplification_tolerance=None,
    overrides=""
)